In [10]:
import numpy as np
import math
from tabulate import tabulate
from chipsplitting import pairing_matrix
from chipsplitting.hyperfield import HyperfieldVector as HV, HyperfieldPascalForm as HVPascal, HyperfieldHomogeneousLinearSystem as HVLinearSystem, grid_iter
from chipsplitting.utils import gauss, ncr

### Proof of Lemma 6.21 a,b,c

Goal: Find the number of valid configurations $s \in H^{V_d}$ with $|\mathrm{supp}^+(s)| = 4$ such that all basis systems $\varphi, \psi, \bar \psi$ vanish at $s$.

In [11]:
def solve_homogen_linear_system(equations, support_size, mode="plain"):
    solutions = []
    degree = equations[0].degree
    for supp in grid_iter(degree, support_size):
        w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
        w[0] = -1
        v = HV(w)

        if mode == "plain":
            is_valid = True
            for p in equations:
                if not np.isnan(p(v)):
                    is_valid = False
                    break
        
            if is_valid:
                solutions.append(w)
        elif mode == "linear_system":
            P = HVLinearSystem(equations)
            if P.is_solved_by_valid(v):
                solutions.append(w)
    return solutions

In [12]:
%%time
support_size = 4
for degree in range(6, 12):
    pascal_eqns = [HVPascal(degree, mode, unit) for mode in ["row", "diag", "col"] for unit in range(degree + 1)]
    solutions = solve_homogen_linear_system(pascal_eqns, support_size, "plain")
    print(f"Number of configurations for degree={degree}: {len(solutions)}")
    if solutions:
        print(solutions)

Number of configurations for degree=6: 5
[array([-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  1,  1,  0,  0,  1,  0,  0,  0]), array([-1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,
        0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0]), array([-1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  0,  0,  1,  0,  0,  0,  0,  1]), array([-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,
        0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  1]), array([-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  1,  0,  0,  1,  0,  0,  1])]
Number of configurations for degree=7: 3
[array([-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
        0,  1]), array([-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,

In [15]:
%%time
support_size = 5
degree_begin = 8
degree_end = 12
for degree in range(degree_begin, degree_end + 1):
    pascal_eqns = [HVPascal(degree, mode, unit) for mode in ["row", "diag", "col"] for unit in range(degree + 1)]
    solutions = solve_homogen_linear_system(pascal_eqns, support_size, "plain")
    print(f"Number of configurations for degree={degree}: {len(solutions)}")
    # if solutions:
    #   print(solutions)

Number of configurations for degree=8: 792
Number of configurations for degree=9: 882
Number of configurations for degree=10: 950
Number of configurations for degree=11: 1084
Number of configurations for degree=12: 1102
CPU times: user 42min 27s, sys: 13.8 s, total: 42min 40s
Wall time: 42min 42s



__Conclusion__: We conclude that only configurations $s$ of degree $d = 6$ or $d = 7$ belong to $\Omega_d$.

### Lemma 6.21 d

Fix some degree $d$.
We define $\hat \varphi_i, i=1,2,3$ to be the linear form of $\varphi_{i, d - i}$ in $H[x,y,z,b,c,d,e]$. A configuration $s$ in $H^{\Xi}$ uses the following coordinates $s = [ s_{i} ]_{i=0,...,59} = [x_0,...,x_{15}, y_{16},...,y_{31},z_{32},...,z_{47}, b_{48}, ..., b_{51}, c_{52},..., c_{55}, d_{56}, ..., d_{59}, e_{60},...,e_{63}] \in H^{64}$.

#### Find valid configurations for $d \geq 12$

Fix some degree $d$. We define the following sets of Pascal equations 
$$
    A = \{ \psi_1,..., \psi_6, \varphi_1, ...,  \varphi_6 \}
$$
where $\psi_1, \psi_2, \psi_3$ are the Pascal equations respectively defined by 
```
0                0                0
* .              * .              * .
0 . .            0 . .            1 . .
0 . . .          1 . . .          0 . . .
1 . . . .        0 . . . .        0 . . . .
0 . . . . .      0 . . . . .      0 . . . . .       * stands for arbitrarily many 0
```
$\psi_4, \psi_5, \psi_6$ are the Pascal equations respectively defined by 
```
.                .                .
. .              . .              . .
. . .            . . .            . . .
. . . .          . . . .          . . . .
. . . . .        . . . . .        . . . . . 
0 1 0 0 * 0      0 0 1 0 * 0      0 0 0 1 * 0       * stands for arbitrarily many 0
```
$\varphi_1,...,\varphi_6$ are the Pascal equations respectively defined by
```
0                0                0
. 1              . 0              . 0
. . 0            . . 1            . . 0
. . . 0          . . . 0          . . . 1
. . . . *        . . . . *        . . . . *
. . . . . 0      . . . . . 0      . . . . . 0      

0                0                0
. *              . *              . *
. . 1            . . 0            . . 0
. . . 0          . . . 1          . . . 0
. . . . 0        . . . . 0        . . . . 1
. . . . . 0      . . . . . 0      . . . . . 0       * stands for arbitrarily many 0
```
and $\xi_1,...,\xi_8$ are the Pascal equations respectively defined by
```
1                 0                0                0
0 .               1 .              0 .              0 .
0 . .             0 . .            1 . .            0 . .
0 . . .           0 . . .          0 . . .          1 . . .
* . . . .         * . . . .        * . . . .        * . . . . 
0 . . . . .       0 . . . . .      0 . . . . .      0 . . . . .      

.                .                .                .
. .              . .              . .              . .
. . .            . . .            . . .            . . .
. . . .          . . . .          . . . .          . . . .
. . . . .        . . . . .        . . . . .        . . . . . 
0 * 0 0 0 1      0 * 0 0 1 0      0 * 0 1 0 0      0 * 1 0 0 0       * stands for arbitrarily many 0
```

In [4]:
def eval_f(support,x):
    support_pos, support_neg = support[0], support[1]
    if any(np.isnan(x[support_pos | support_neg])):
        return np.nan
    has_neg = any(x[support_pos] < 0) | any(x[support_neg] > 0)
    has_pos = any(x[support_pos] > 0) | any(x[support_neg] < 0)
    if has_neg and has_pos:
        return np.nan
    if has_neg:
        return -1
    elif has_pos:
        return 1
    return 0

In [5]:
%%time
def countValidConfigsForContractions(degree="even"):
    assert degree == "even" or degree == "odd", "degree must be 'even' or 'odd'"
    
    def create_hat_exhaust_generator():
        # for d in range(63, 15, -1):
        for d in [19,23,27,31,56,60,47,46,45,44]:
            for i in range(1, 64):
                if i == d:
                    continue
                for j in range(1, i):
                    if j == d:
                        continue
                    for k in range(1, j):
                        if k == d:
                            continue
                        yield (d, i, j, k)
                    
    res = []
    A = np.array([
        PSI_HAT_1_SUPP, PSI_HAT_2_SUPP, PSI_HAT_3_SUPP, 
        PSI_HAT_4_SUPP, PSI_HAT_5_SUPP, PSI_HAT_6_SUPP, 
        PHI_HAT_1_SUPP, PHI_HAT_2_SUPP, PHI_HAT_3_SUPP, 
        PHI_HAT_4_SUPP, PHI_HAT_5_SUPP, PHI_HAT_6_SUPP
    ])
    EVEN = np.array([
        XI_HAT_1_EVEN_SUPP, XI_HAT_2_EVEN_SUPP, XI_HAT_3_EVEN_SUPP, 
        XI_HAT_4_EVEN_SUPP, XI_HAT_5_EVEN_SUPP, XI_HAT_6_EVEN_SUPP,
        XI_HAT_7_EVEN_SUPP, XI_HAT_8_EVEN_SUPP
    ])
    ODD = np.array([
        XI_HAT_1_ODD_SUPP, XI_HAT_2_ODD_SUPP, XI_HAT_3_ODD_SUPP, 
        XI_HAT_4_ODD_SUPP, XI_HAT_5_ODD_SUPP, XI_HAT_6_ODD_SUPP,
        XI_HAT_7_ODD_SUPP, XI_HAT_8_ODD_SUPP
    ])

    for supp in create_hat_exhaust_generator():
        w = np.array([1 if idx in supp else 0 for idx in range(64)])
        w[0] = -1
        
        is_valid = True
        for f in np.concatenate([A, EVEN if degree == "even" else ODD]):
            val = eval_f(f, w)
            if not np.isnan(val):
                is_valid = False
                break
    
        if is_valid:
            res.append(w)

    return res

res1 = countValidConfigsForContractions("even")
print(f"Number of configurations for even d: {len(res1)}")

res2 = countValidConfigsForContractions("odd")
print(f"Number of configurations for odd d: {len(res2)}")

Number of configurations for even d: 0
Number of configurations for odd d: 0
CPU times: user 11.5 s, sys: 31.7 ms, total: 11.5 s
Wall time: 11.5 s


**Conclusion:** For $d \geq 12$ there are no valid configurations in $H^{64}$ that satisfy all Pascal equations in $A \cup \mathrm{EVEN}$ or $A \cup \mathrm{ODD}$.

**Why is this good?** It is a necessary condition for valid configurations $w \in \mathbb Z^{V_d}$ that all Pascal equations vanish at $\mathrm{sign}(w)$. Since $A \cup \mathrm{EVEN}$ is a subset of all Pascal equations and $A \cup \mathrm{EVEN}$ have no zeros for configurations $s$ with support size $\leq 4$, no valid $w \in \mathbb Z^{V_{d}}$ exist for $d \geq 12$.

### Conclusion

With Lemma 6.21 a,b,c,d we see that valid configurations $s \in H^{V_d}$ with support size $4$ do not exist for degree $d \geq 8$. The next step is to show that for $d = 6$ and $d = 7$, the valid configurations $s \in H^{V_d}$ do not correspond to valid configurations $w \in \mathbb Z^{V_d}$. Thus, after showing this we can conclude that $|\mathrm{supp}^+(w)| = 4 \implies \mathrm{deg}(w) \leq 5$.

## Theorem 6.20

We now prove that outcomes with support size $4$ are at most of degree $5$.

From the previous sections, we already concluded that outcomes with support size $4$ are at most of degree $7$. We now reduce the degree bound to $5$. 

For degree $6$ we computed the following valid configurations in the hyperfield. We need to check if they are valid outcomes in $\mathbb Z^{V_6}$.

```
[array([-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  1,  1,  0,  0,  1,  0,  0,  0]),
 array([-1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0]),
 array([-1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  1,  0,  0,  1,  0,  0,  0,  0,  1]),
 array([-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  1]),
 array([-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  1,  0,  0,  1])]
```

In [24]:
degree = 6
configurations = [
    [-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  1,  1,  0,  0,  1,  0,  0,  0],
    [-1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0],
    [-1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  1,  0,  0,  1,  0,  0,  0,  0,  1],
    [-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  1],
    [-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  1,  0,  0,  1]
]

for row in configurations:
    support = [toIndex(i) for i, x in enumerate(row) if x == 1 or x == -1]
    basis = list(range(len(support)))
    P = pairing_matrix(degree, basis, support)
    print(f"Pairing matrix (support={support}, basis={basis}) has determinant: {np.linalg.det(P)}")

Pairing matrix (support=[(0, 0), (1, 1), (5, 0), (0, 6), (3, 3)], basis=[0, 1, 2, 3, 4]) has determinant: 0.0
Pairing matrix (support=[(0, 0), (3, 0), (1, 4), (0, 6), (5, 1)], basis=[0, 1, 2, 3, 4]) has determinant: 0.0
Pairing matrix (support=[(0, 0), (0, 3), (4, 1), (1, 5), (6, 0)], basis=[0, 1, 2, 3, 4]) has determinant: 0.0
Pairing matrix (support=[(0, 0), (1, 1), (0, 5), (3, 3), (6, 0)], basis=[0, 1, 2, 3, 4]) has determinant: 0.0
Pairing matrix (support=[(0, 0), (1, 1), (0, 6), (3, 3), (6, 0)], basis=[0, 1, 2, 3, 4]) has determinant: 0.0


**Conclusion:** By the intervertibility criterion only the initial configuration $\mathbf 0 \in \mathbb Z^{V_d}$ can live in the support given above. Thus, no outcome of degree $6$ with support size $4$ exist.

For degree $7$ we have the following support set that we need to check
```
[array([-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
         0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  1]),
 array([-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  1]),
 array([-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,
         0,  1])]
```

In [25]:
degree = 7
configurations = [
    [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
         0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  1],
    [-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  1],
    [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,
         0,  1]
]

for row in configurations:
    support = [toIndex(i) for i, x in enumerate(row) if x == 1 or x == -1]
    basis = list(range(len(support)))
    P = pairing_matrix(degree, basis, support)
    print(f"Pairing matrix (support={support}, basis={basis}) has determinant: {np.linalg.det(P)}")

Pairing matrix (support=[(0, 0), (3, 1), (1, 5), (0, 7), (7, 0)], basis=[0, 1, 2, 3, 4]) has determinant: 0.0
Pairing matrix (support=[(0, 0), (1, 1), (3, 3), (0, 7), (7, 0)], basis=[0, 1, 2, 3, 4]) has determinant: 0.0
Pairing matrix (support=[(0, 0), (1, 3), (5, 1), (0, 7), (7, 0)], basis=[0, 1, 2, 3, 4]) has determinant: 0.0


**Conclusion:** By the intervertibility criterion only the initial configuration $\mathbf 0 \in \mathbb Z^{V_d}$ can live in the support given above. Thus, no outcome of degree $7$ with support size $4$ exist.

### Conclusion

This proves Theorem 6.20.